In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Import dataset
df = pd.read_csv('../data/base_modelis_pass_det.csv',sep = ';')
df2 = pd.read_csv('../data/base_modelis_pass_det.csv',sep = ';')

liste_var = [#'mt_dec_cum_IMP_tot','EAD_1an_avant_def','cum_frs_externes','fl_prise_de_gar', 'cum_rec', 'fam_ETH', 'CD_DNE_CHR_LGD_SAIN','CD_MTF_ENE_CTX',
             # anciennes variables ->
             'CD_NATUR_OP_PFI', 'CD_SITFAM_EMP1', 'sol_echec', 'CD_MTF_DFC', 'regroup_natur_op', 'fam_PCD', 'fl_prise_de_gar', 'mt_dec_cum_IMP_tot', 'dat_dec_echec', 'fam_ENQ',
 'niv_gravite_mtf_dne', 'fl_fam_vam', 'fl_pass_DET', 'arrete', 'cd_op']



liste_var = ['AMI', 'regroup_natur_op' , 'fl_fonc', 'fl_prt_Conso', 'nb_empr_ope' , 'niv_gravite_mtf_dfs', 'CD_MTF_DFC', 'sol_echec', 'fam_PCD', 'solution', 'dat_dec_echec', 'NB_RP_HOP', 'CD_DNE_CHR_LGD_SAIN', 'CD_SITFAM_EMP1', 'CD_CSP_EMP1', 'CD_ETAT_CIVIL', 'CD_NATUR_OP_PFI', 'MT_CHA_CSM_REV', 'MT_CHA_IMP', 'MT_CHA_PIP', 'MT_CHA_PIV', 'mt_rev_dispo', 'MT_CHG_MSL', 'MT_REST_VIVRE', 'ANC_BANC_MAX_PFI', 'TX_ENDET', 'CD_MTF_ENE_CTX', 'MT_PATRIM_NET', 'ANC_PRO_MAX_PFI', 'MT_CHA_HORS_OPE_PFI', 'NB_CHARGE_HORS_OPE_PFI', 'max_age_ctx', 'EAD_1an_avant_def', 'MT_REV_PFI', 'TX_APPOR', 'cum_frs_externes', 'MT_PATRIM_MOB', 'AGE_EMP1', 'dls_ges_def', 'NB_DUR_PFI', 'mt_dec_cum_IMP_tot', 'MT_FMG', 'MT_EAD_RESID', 'cum_dec', 'nb_bienHOP', 'dls_mep_entree_def', 'CD_POST_BIEN_PFI', 'cum_rec', 'MT_EAD_DNE_CHR_d', 'MR_arr', 'NB_LO_HOP','fl_pass_DET', 'arrete', 'cd_op']

liste_var = list(set(liste_var))
# Variables selctionnées
df = df[liste_var]#'CD_NAT_EMP1', 'CD_NAT_EMP2','CD_CSP_EMP2' trop de NaN pour ces var ,'CD_DEST_PFI',  'CD_CSP_EMP1'

# Peut etre drop CSP et DEST car on peut pas regrouper les categories

'MT_CHA_PIP', 'FL_ETR', 'cum_frs_externes', 'MT_EAD_RESID',
       'fl_prise_de_gar', 'fam_ETH', 'fam_ENQ', 'fam_hyp', 'dat_dec_echec',
       'solution', 'fam_PCD', 'dtSRT1st', 'CD_MTF_ENE_CTX',
       'niv_gravite_mtf_dne', 'niv_gravite_mtf_dfs', 'dls_mep_entree_def',
       'nb_bienHOP', 'ANC_PRO_MAX_PFI', 'nb_empr_ope', 'fl_def_ss_pass_CTX',
       'mt_dec_cum_IMP_tot', 'AMI', 'PTG', 'AGE_EMP2',

# DataTreatment

In [ ]:
filtre = (df.select_dtypes(include = 'number').nunique()<10)
for col in df.select_dtypes(include = 'number').loc[:,filtre].columns:
    df[col] = df[col].astype('object')

In [ ]:
df['arrete'] = pd.to_datetime(df['arrete'])

In [ ]:
liste_det = df[df['fl_pass_DET'] == 1]['cd_op']

# on applique le det ou non à toutes les lignes des indiv pour ne garder que la première ligne par individu et garder l'info du passage en DET ou non

df['DET'] = np.where(df['cd_op'].isin(liste_det), 1, 0)

In [ ]:
liste_det = df2[df2['fl_pass_DET'] == 1]['cd_op']

# on applique le det ou non à toutes les lignes des indiv pour ne garder que la première ligne par individu et garder l'info du passage en DET ou non

df2['DET'] = np.where(df2['cd_op'].isin(liste_det), 1, 0)

Ensuite on va supprimer les individus qui n'ont qu'une ligne dans le dataframe car ce sont des anomalies.

In [ ]:
anomalie = (df['cd_op'].value_counts() ==1)
anomalie = pd.DataFrame(anomalie)
liste_anomalie = anomalie[anomalie['count']==True].index

In [ ]:
df['anomalie'] = np.where(df['cd_op'].isin(liste_anomalie), 1, 0)

In [ ]:
df = df[df['anomalie'] == 0]

In [ ]:
df.drop(columns = ['fl_pass_DET','anomalie'], inplace = True)

In [ ]:
df.drop_duplicates(subset = 'cd_op', inplace = True , keep = 'first')

In [ ]:
df['DET'].value_counts()

In [ ]:
df.shape

# NaNs

In [ ]:
df.isna().sum()

In [ ]:
for col in df.select_dtypes(include = 'number').columns:
    valeur = df[col].median()
    df[col].fillna(valeur, inplace=True)
    
    
for col in df.select_dtypes(include = 'object').columns:
    valeur = df[col].mode()
    df[col].fillna(valeur.iloc[0], inplace=True)

In [ ]:
df.isna().sum().sum()

# Regroupement de categories 

### Motif

In [ ]:
motif_tres_mauvais = ['ESC' , 'REL', 'ORG', 'PPE', 'DEC', 'SIN']
motif_mauvais = ['MOR', 'CHO', 'DIV', 'SUR', 'RIS', 'SEP', 'SAN', 'CON', 'DI1' ]  
motif_moyen = ['BDF', 'AUT', 'PCL', 'PCT', 'YYY']

df['Motif'] = np.select(
    [
        df['CD_MTF_DFC'].isin(motif_tres_mauvais),
        df['CD_MTF_DFC'].isin(motif_mauvais),
        df['CD_MTF_DFC'].isin(motif_moyen)
    ],
    ['Motif tres mauvais', 'Motif mauvais', 'Motif moyen'],
    default='Motif moyen')
df.drop(columns ='CD_MTF_DFC', inplace = True)

# Gravite

In [ ]:
precaire = [2.0,3.0]
df['niv_gravite_mtf_dfs'] = np.where(df['niv_gravite_mtf_dfs'].isin(precaire), 'Grave', 'Pas grave')

### CD_SITFAM_EMP1

In [ ]:
precaire = ['I','D','C'] 

df['Situation_fam'] = np.where(df['CD_SITFAM_EMP1'].isin(precaire), 'Mauvaise situation', 'Bonne situation')

df.drop(columns ='CD_SITFAM_EMP1', inplace = True)

### CD_NATUR_OP_PFI

In [ ]:
precaire = ['ACR', 'ARA', 'SCP', 'ATR', 'COR', 'AQN', 'AQA']

df['Nature'] = np.where(df['CD_NATUR_OP_PFI'].isin(precaire), 'Mauvaise nature', 'Bonne nature')

df.drop(columns ='CD_NATUR_OP_PFI', inplace = True)

## Sol_echec

In [ ]:
precaire = ['0_1', '1_1']
# Ajouter une colonne 'Groupe' au DataFrame en fonction des catégories
df['Solution_echec'] = np.where(df['sol_echec'].isin(precaire), 'Mauvaise solution', 'Bonne solution')

df.drop(columns ='sol_echec', inplace = True)

## Nature operation

In [ ]:
precaire = ['4_autre']

df['Nature_op'] = np.where(df['regroup_natur_op'].isin(precaire), 'Mauvaise nature', 'Bonne nature')

df.drop(columns ='regroup_natur_op', inplace = True)

# MTF

In [ ]:
precaire = [102.0]
df['MTF'] = np.where(df['CD_MTF_ENE_CTX'].isin(precaire), 'Mauvais MTF', 'Bon MTF')
df.drop(columns ='CD_MTF_ENE_CTX', inplace = True)

# ETAT_CIVIL

In [ ]:
precaire = [4.0,5.0,2.0,8.0,3.0,7.0]
df['ETAT_CIVIL'] = np.where(df['CD_ETAT_CIVIL'].isin(precaire), 'Mauvais etat', 'Bon etat')

df.drop(columns ='CD_ETAT_CIVIL', inplace = True)

# CSP

In [ ]:
precaire = ['PEO' , 'PAM', 'FEO', 'FCA']
# Ajouter une colonne 'Groupe' au DataFrame en fonction des catégories
df['CSP'] = np.where(df['CD_CSP_EMP1'].isin(precaire), 'Mauvaise CSP', 'Bonne CSP')

df.drop(columns ='CD_CSP_EMP1', inplace = True)

# Index

In [ ]:
df.index = df['cd_op']
df.drop(columns = 'cd_op',inplace = True)

# Encoding

### Label encoding pour modalités ayant une relation ordinale

In [ ]:
df['niv_gravite_mtf_dfs'] = df['niv_gravite_mtf_dfs'].apply(lambda x : 1 if x == 'Grave' else 0)
df['niv_gravite_mtf_dfs'] = df['niv_gravite_mtf_dfs'].astype('int')

# OHE pour le reste

for col in df.select_dtypes(include = 'object').columns:
    dummies_df = pd.get_dummies(df[col], prefix=f'{col}')
    
    df = pd.concat([df, dummies_df], axis=1)
    
    df.drop(columns = col, inplace = True)

Il faudra penser à drop une colonne pour chaque variable categorielle pour qu'elle soit considérée comme modalité de reference pour eviter les pb de multicolinéarité.

# Var numériques

In [ ]:
for col in df.select_dtypes(include = 'bool').columns : 
    df[col] = df[col].apply(lambda x: 1 if x == True else 0)

# Export du dataframe 

In [ ]:
df.to_csv('../data/df.csv', sep = ',')

# Discretisation var num

In [ ]:
df2.drop_duplicates(subset = 'cd_op', inplace = True, keep = 'first')

In [ ]:
import matplotlib.pyplot as plt

df_det = df2[df2['DET'] == 1].iloc[:500,:]
df_cool = df2[df2['DET'] == 0].iloc[:500,:]
df3 = pd.concat([df_det,df_cool])

for col in df3.select_dtypes(include = 'number').columns : 
    if df3[col].nunique()>1 :
        colors = {1 : 'red', 0 : 'blue'}
        df3['Couleur'] = df3['DET'].map(colors)
        
        # Scatter plot
        plt.scatter(df3[col], range(len(df3)), c=df3['Couleur'], label=df3['DET'])
        
        # Ajouter des labels et un titre
        plt.xlabel('Variable Quantitative')
        plt.ylabel('Index')
        plt.title(col)
        
        # Afficher le plot
        plt.show()
    else:
        pass

In [ ]:
df2['MT_REST_VIVRE'].isna().sum()